In [3]:
from pprint import pprint

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, match_dummies_to_varNames, joining_variablesDict_onCol
from python_lib.HPDS_connection_manager import tokenManager

In [4]:
# Pandas DataFrame display options
pd.set_option("max.rows", 435)

# Matplotlib display parameters
plt.rcParams["figure.figsize"] = (14,8)
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

In [5]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"
token = tokenManager(token_file).get_token()


Security Token Imported Correctly


In [6]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, token)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

## Get studies information

In [63]:
from ast import literal_eval
studies_info = pd.read_csv("../studies_info.csv",
                           index_col=0, 
                          converters={"phs_list": literal_eval})

## Get individual studies statistics

In [17]:
studies_info

,phs,BDC_study_name,official_study_name,short_name,harmonized,list_phs,ID_varName
0,phs000007,Framingham Cohort,Framingham Cohort,FHS,True,"['phs000007.c0', 'phs000007.c1', 'phs000007.c2']",\Framingham Cohort\Administration\SHARe IDs: a...
1,phs000179,Genetic Epidemiology of COPD (COPDGene),Genetic Epidemiology of COPD (COPDGene) Funded...,COPDGene,True,"['phs000179.c0', 'phs000179.c1', 'phs000179.c2']",\Genetic Epidemiology of COPD (COPDGene)\Sampl...
2,phs000200,Women's Health Initiative,Women's Health Initiative Clinical Trial and O...,WHI,True,"['phs000200.c0', 'phs000200.c1', 'phs000200.c2']",\Women's Health Initiative\Subject Sample Mapp...
3,phs000209,Multi-Ethnic Study of Atherosclerosis (MESA) C...,Multi-Ethnic Study of Atherosclerosis (MESA) C...,MESA,True,"['phs000209.c0', 'phs000209.c1', 'phs000209.c2']",\Multi-Ethnic Study of Atherosclerosis (MESA) ...
4,phs000280,Atherosclerosis Risk in Communities (ARIC) Cohort,Atherosclerosis Risk in Communities (ARIC) Cohort,ARIC,True,"['phs000280.c0', 'phs000280.c1', 'phs000280.c2']",\Atherosclerosis Risk in Communities (ARIC) Co...
5,phs000284,NHLBI Cleveland Family Study (CFS) Candidate G...,NHLBI Cleveland Family Study (CFS) Candidate G...,CFS,True,"['phs000284.c0', 'phs000284.c1']",\NHLBI Cleveland Family Study (CFS) Candidate ...
6,phs000285,NaN,Coronary Artery Risk Development in Young Adul...,CARDIA,NaN,"['phs000285.c1', 'phs000285.c2']",NaN
7,phs000286,The Jackson Heart Study (JHS),Jackson Heart Study (JHS) Cohort,JHS,True,"['phs000286.c0', 'phs000286.c1', 'phs000286.c2...",\The Jackson Heart Study (JHS)\Subjects\Subjec...
8,phs000287,Cardiovascular Health Study (CHS) Cohort,Cardiovascular Health Study (CHS) Cohort: an N...,CHS,False,"['phs000287.c0', 'phs000287.c1', 'phs000287.c2...",\Cardiovascular Health Study (CHS) Cohort\CARe...
9,phs000289,NHGRI Genome-Wide Association Study of Venous ...,National Human Genome Research Institute (NHGR...,Mayo_VTE,False,"['phs000289.c0', 'phs000289.c1']",\NHGRI Genome-Wide Association Study of Venous...


In [22]:
plain_variablesDict = resource.dictionary().find().DataFrame()

In [36]:
mask_consent_var = plain_variablesDict.index.str.contains("Study Accession with Consent Code")
consent_var = plain_variablesDict.index[mask_consent_var].values[0]

In [73]:
#for phs in phs_index:
def study_variables(phs, studies_info, consent_var, resource):
    phs_list = studies_info.loc[phs, "phs_list"]
    query = resource.query()
    query.filter().add(consent_var, phs_list)
    query.select().add(plain_variablesDict.index.tolist())
    facts = query.run().getResultsDataFrame()
    return facts

In [65]:
phs = studies_info.index[1]

In [ ]:
facts = study_variables(phs, studies_info, consent_var, resource)